# Sight Lines computation

In [1]:
import arcpy

arcpy.env.workspace = "C:/Users/g_filo01/sciebo/Scripts/Image of the City/Outputs/tmp"
env = arcpy.env.workspace
arcpy.CheckOutExtension("3D")
arcpy.env.overwriteOutput = True

In [2]:
# output folder from which loading
city_name = 'Boston'
load_folder = "C:/Users/g_filo01/sciebo/Scripts/Image of the City/Outputs/"+city_name

In [3]:
observer_points = load_folder+"/"+city_name+"_copy.shp"
obstructions_file = env+"/"+city_name+"_obstructions.shp"
buildings = city_name+"_sight.shp"
buildings3d = "buildings3d"

sightlines_file = city_name+"_sightlines.shp"
height_observer = "height"
heigth_buildings = "height"
direction = "NOT_OUTPUT_THE_DIRECTION"

In [5]:
aprx = arcpy.mp.ArcGISProject("C:/Users/g_filo01/sciebo/Scripts/ArcGis/empty.aprx")
buildings_layer = arcpy.MakeFeatureLayer_management(buildings, "as_layer").getOutput(0)
layer3d = arcpy.FeatureTo3DByAttribute_3d(buildings_layer, "layer3d", "base", to_height_field = "height").getOutput(0)
# temp = arcpy.AddZInformation_3d(layer3d, 'Z_MAX', 'NO_FILTER').getOutput(0)
layer = arcpy.MakeFeatureLayer_management(layer3d, "tmp_layer").getOutput(0)

In [6]:
m = aprx.listMaps()[0]
m.addLayer(layer)

for ma in aprx.listMaps():
    for lyr in ma.listLayers():
        print(lyr)
        lyr.extrusion("BASE_HEIGHT", "height")

tmp_layer


In [ ]:
# obstructions = arcpy.Layer3DToFeatureClass_3d(m.listLayers()[0],
#                                               "obs", None, "ENABLE_COLORS_AND_TEXTURES").getOutput(0)

In [ ]:
obstructions = arcpy.Layer3DToFeatureClass_3d(m.listLayers()[0], "obs", None, "ENABLE_COLORS_AND_TEXTURES").getOutput(0)
layer = arcpy.MakeFeatureLayer_management(obstructions, "obstructions_layer").getOutput(0)
m.addLayer(layer)

In [ ]:
# for ma in aprx.listMaps():
#     for lyr in ma.listLayers():
#         print(lyr)
#         lyr.extrusion("ABSOLUTE_HEIGHT", "Z_MAX")

In [7]:
aprx.saveACopy(env+"/copy.aprx")

In [ ]:
geoDB = "C:/Users/g_filo01/sciebo/Scripts/ArcGis/GeoDB.gdb"
obstructions = geoDB+"/"+city_name+"_obstructions"

In [ ]:
geoDB = "C:/Users/g_filo01/sciebo/Scripts/ArcGis/GeoDB.gdb"
obstructions = geoDB+"/"+city_name+"_obstructions
arcpy.ddd.ConstructSightLines(observer_points, buildings, sightlines_file, 
                              height_observer, heigth_buildings, None, 30, direction)
with arcpy.da.UpdateCursor(sightlines_file, 'SHAPE@LENGTH') as cursor:
    for row in cursor:
        if row[0] < 100:
            cursor.deleteRow()

In [ ]:
with arcpy.da.UpdateCursor(sightlines_file, 'SHAPE@LENGTH') as cursor:
    for row in cursor:
        if row[0] < 100:
            cursor.deleteRow()

In [ ]:
obstructions_file = env+"/"+city_name+"_obstructions.shp"

obstructions = geoDB+"/"+city_name+"_obstructions"

In [ ]:
arcpy.ddd.Intervisibility(sightlines_file, obstructions = obstructions, visible_field="Visibility")

In [ ]:
# arcpy.FeatureClassToShapefile_conversion(sightlines_file, arcpy.env.workspace)

In [ ]:
with arcpy.da.UpdateCursor(sightlines_file, "Visibility") as cursor:
    for row in cursor:
        if row[0] == 0: 
            cursor.deleteRow()

In [ ]:
arcpy.JoinField_management(sightlines_file, "OID_OBSERV", observer_points, "FID", "nodeID")
arcpy.JoinField_management(sightlines_file, "OID_TARGET", buildings, "FID", "buildingID")
arcpy.DeleteField_management(sightlines_file, ["OID_OBSERV", "OID_TARGET"])

In [ ]:
arcpy.FeatureClassToShapefile_conversion(sightlines_file, arcpy.env.workspace)